In [ ]:
# Change an input here:

INPUT_FILE = '' # '/eos/cms/store/group/dpg_trigger/comm_trigger/L1Trigger/bundocka/EZB_12_108p1/EphemeralZeroBias3/zbD3/211125_213433/0000/L1Ntuple_10.root'
MODEL_PATH = '' #'ML_CMSL1CaloTrigger/saved_models/qmodel'

In [ ]:
import awkward as ak
import numpy as np
import tensorflow
import uproot

from dataclasses import dataclass
from skimage.measure import block_reduce
from qkeras.utils import load_qmodel

In [ ]:
@dataclass
class DataSource:
    ids: ak.highlevel.Array
    phi: ak.highlevel.Array
    eta: ak.highlevel.Array
    et: ak.highlevel.Array
    size: int
    _calo_vars = ['iet', 'ieta', 'iphi', 'iem']

    def __init__(self, input_file, tree_name):
        with uproot.open(input_file) as in_file:
            tree = in_file[tree_name]
            arrays = tree.arrays(self._calo_vars)
            eta = arrays['ieta']
            phi = arrays['iphi']
            et = arrays['iet']
            em = arrays['iem']
            self.size = len(eta)
    
            mask = (eta >= -28) & (eta <= 28)
            eta, phi, et, em = eta[mask], phi[mask], et[mask], em[mask]
            eta = ak.where(eta < 0, eta, eta - 1)
            eta = eta + 28
            phi = (phi + 2) % 72
    
            ids = np.arange(len(eta))
            self.ids = ak.flatten(ak.broadcast_arrays(ids, eta)[0])
            self.phi = ak.flatten(phi, None)
            self.eta = ak.flatten(eta, None)
            self.et = ak.flatten(et, None)
    
    def __len__(self):
        return self.size
    
source = DataSource(INPUT_FILE, 'l1CaloTowerEmuTree/L1CaloTowerTree')
events = len(source)
deposits = np.zeros((events, 72, 56))
deposits_ecal = np.zeros((events, 72, 56))

# Get raw data
ids = source.ids.to_numpy()
phi = source.phi.to_numpy()
eta = source.eta.to_numpy()
et = source.et.to_numpy()

# Calculate regional deposits
deposits[ids, phi, eta] = et

# Reduce to towers
region_et = block_reduce(deposits, (1, 4, 4), np.sum)

student = load_qmodel(MODEL_PATH)

In [ ]:
in_file = uproot.open(INPUT_FILE)

for event_id in range(5):
    print("Run: {}. Lumi: {}. Event ID: {}. Anomaly score: {}.".format(
        in_file['l1EventTree/L1EventTree']['run'].array()[event_id],
        in_file['l1EventTree/L1EventTree']['lumi'].array()[event_id],
        in_file['l1EventTree/L1EventTree']['event'].array()[event_id],
        student.predict(np.expand_dims(region_et[event_id], 0)).flatten()[0]
    ))